In [17]:
import pandas as pd
import mat73
from datetime import datetime

In [2]:
df = pd.read_excel('tables/batch1.xlsx')
df = df.rename({'file':'HashFolderName'},axis=1)

In [3]:
df_brandon = pd.read_csv('EEGs_And_Reports_20231024.csv')

In [4]:
df=df.merge(df_brandon[['HashFolderName','SiteID','SessionID_new','EEGFolder']],on='HashFolderName')

In [9]:
root = 'bdsp/opendata/EEG/data/'
i= 0
x = df.iloc[i].copy()
path = f'{x.SiteID}/{x.EEGFolder}/{x.HashFolderName}/'
filename = '_'.join([x.HashFolderName.split('_')[0],'0']+x.HashFolderName.split('_')[1:3])+'.mat'

In [10]:
root = 'bdsp/opendata/EEG/data/'
i= 0
x = df.iloc[i].copy()
path = f'{x.SiteID}/{x.EEGFolder}/{x.HashFolderName}/'
filename = '_'.join([x.HashFolderName.split('_')[0],'0']+x.HashFolderName.split('_')[1:3])+'.mat'

signal = mat73.loadmat(root+path+filename)
data=signal['data']
Fs=signal['Fs']

In [14]:
# Combine date and time into a datetime object
def get_time_from_filename(filename):
    # filename in format blabla_y,m,d_h,m,s.mat
    # returns timestamp obect
    date_str = filename.replace('.mat','').split('_')[-2]
    time_str = filename.replace('.mat','').split('_')[-1]

    date_object = datetime.strptime(date_str, '%Y%m%d')
    time_object = datetime.strptime(time_str, '%H%M%S').time()

    # Combine date and time into a datetime object
    start = datetime.combine(date_object.date(), time_object)   
    return start

In [15]:
def get_event_snippet(data,event_time,windowsize):
    # takes a piece of signal of shape [channel, ts], the timestamp of the event and the windowsize
    # returns the event pm windowsize, shape [channel, windowsize]
    snippet_start = int((event_time-windowsize/2)*Fs)
    snippet_end = int((event_time+windowsize/2)*Fs)
    return data[:,snippet_start:snippet_end]

In [16]:
relative_start_time = get_time_from_filename(filename)
relative_event_time = x.time
absolute_event_time = (relative_event_time-relative_start_time).total_seconds()
windowsize = 15
snippet = get_event_snippet(data=data,event_time=absolute_event_time,windowsize=windowsize)


AttributeError: module 'datetime' has no attribute 'strptime'